# Mapping Vehicle Ownership
### Directions: run each cell to generate an interactive map of census tracts shaded by the poverty rate
#### There are drop-down menus to select geographies
#### The italicized text explains the code in the box

In [5]:
# remove the "#" below and run this cell if you get an error running the cell below to set up the work environment
# !pip install mapboxgl

In [2]:
# set up work environment
# if you get an error about missing mapboxgl, then go back and follow the instructions in the first cell
%matplotlib inline
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
from scipy import ndimage
import matplotlib.pylab as pylab
import json
import requests
import pprint
import numpy as np
from mapboxgl.viz import *
from mapboxgl.utils import *
import ipywidgets as widgets

pylab.rcParams['figure.figsize'] = 10, 8

import warnings 
warnings.filterwarnings('ignore')

In [3]:
# mapbox token
token = 'pk.eyJ1IjoicmFjaGVsb20iLCJhIjoiY2puYXE2YnV4NTVmcjNxbjE2ZHdjazVlbCJ9.jZKiF7-Y9c1gIGDvO0SmwA'

### Calculate the Percent of Households with No Vehicle

In [4]:
# fips links to a table that includes fips codes for all states and counties in the United States
fips = pd.read_csv('https://raw.githubusercontent.com/omrachel/equitymetrics/master/tables/FIPS_Code_Dictionary.txt', header=None,
                   names=['State Name','State Code', 'County Code', 'County Name','Code'],dtype=str,
                   usecols=['State Name','State Code', 'County Code', 'County Name'])

In [5]:
# Select the year:
pick_year = widgets.Dropdown(options=['Select Year', '2016', '2015', '2014'],value='Select Year',description='Year:',disabled=False)
display(pick_year)

Dropdown(description='Year:', options=('Select Year', '2016', '2015', '2014'), value='Select Year')

In [6]:
# Select the state
state_list = list(fips['State Name'].unique())
pick_state = widgets.Dropdown(options=state_list,value='AL',description='State:',disabled=False)
display(pick_state)

Dropdown(description='State:', options=('AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI'…

In [7]:
# Select the county
county_list = list(fips['County Name'])
pick_county = widgets.Dropdown(options=county_list,value='Autauga County',description='County:',disabled=False)
display(pick_county)

Dropdown(description='County:', options=('Autauga County', 'Baldwin County', 'Barbour County', 'Bibb County', …

In [8]:
# visualize poverty rate by census tract
# set up the api
endpoint = 'https://api.census.gov/data/'
# the year is set by the drop down menu above
year = pick_year.value
# can replace with another dataset: https://api.census.gov/data.html
dataset = 'acs/acs5'
# can replace with other variables: https://api.census.gov/data/2016/acs/acs5/variables.html
tables = 'B08201_001E,B08201_002E'
# the state is set by the drop down menu above
state = fips.loc[fips['State Name'] == pick_state.value, 'State Code'].values[0]
# can replace with other county codes: https://www.census.gov/geo/reference/codes/cou.html
county = fips.loc[fips['County Name'] == pick_county.value, 'County Code'].values[0]
# can replace with another key: https://api.census.gov/data/key_signup.html
key = '743d7a3fd3ab38085c25887ef6bed9a331092929'

# this query gets data for the specified year, dataset, and tables for ALL census tracts in the specified state and county
url = requests.Request('GET', endpoint+year+'/'+dataset+'?get=NAME,'+tables+\
                       '&for=tract:*&in=state:'+state+'&in=county:'+county+'&key='+key).prepare().url

print(url)

https://api.census.gov/data/2016/acs/acs5?get=NAME,B08201_001E,B08201_002E&for=tract:*&in=state:06&in=county:037&key=743d7a3fd3ab38085c25887ef6bed9a331092929


In [9]:
response = requests.get(url)
results = response.text
# view first 500 characters of query to make sure data is correct
print(results[:500])

[["NAME","B08201_001E","B08201_002E","state","county","tract"],
["Census Tract 1011.10, Los Angeles County, California","1634","100","06","037","101110"],
["Census Tract 1011.22, Los Angeles County, California","1242","0","06","037","101122"],
["Census Tract 1012.10, Los Angeles County, California","2304","253","06","037","101210"],
["Census Tract 1012.20, Los Angeles County, California","1220","154","06","037","101220"],
["Census Tract 1013, Los Angeles County, California","1461","72","06","037


In [10]:
data = json.loads(results)
# convert data to dataframe so we can analyze

# create list of columns to use
columns = ["Description", "TotalPop","Zero_Vehicle","state", "county", "tract"]

# create dataframe from data and drop first row
zero_vehicle =pd.DataFrame.from_records(data,columns=columns).drop(0)

# view new data frame
zero_vehicle.head()

,Description,TotalPop,Zero_Vehicle,state,county,tract
1,"Census Tract 1011.10, Los Angeles County, Cali...",1634,100,06,037,101110
2,"Census Tract 1011.22, Los Angeles County, Cali...",1242,0,06,037,101122
3,"Census Tract 1012.10, Los Angeles County, Cali...",2304,253,06,037,101210
4,"Census Tract 1012.20, Los Angeles County, Cali...",1220,154,06,037,101220
5,"Census Tract 1013, Los Angeles County, California",1461,72,06,037,101300


In [11]:
# convert numeric columns from string to integer
zero_vehicle[["TotalPop","Zero_Vehicle","tract"]] = zero_vehicle[["TotalPop","Zero_Vehicle","tract"]].astype(int)

# add new column for percent of households in census tract with zero vehicle ownership
zero_vehicle["Zero_Vehicle_Percent"] = round((zero_vehicle["Zero_Vehicle"]/zero_vehicle["TotalPop"]*100),2)

In [12]:
# bring in los angeles county census tracts
la_tracts = gpd.read_file('data/LA_tracts.geojson')
la_tracts.head()

,shape_area,label,x_center,ct10,y_center,geoid10,shape_len,geometry
0,4025735684.42,9110.01,6620403.0,911001,1998891.0,06037911001,353933.808192,(POLYGON ((-117.6671211134298 34.5580081383781...
1,2078689856.02,9800.03,6575300.0,980003,2112006.0,06037980003,273188.86321,(POLYGON ((-117.8806120004425 34.7636159996687...
2,11118018325.1,9303.01,6603027.0,930301,1932124.0,06037930301,628603.531323,(POLYGON ((-117.6552358388393 34.3972219664572...
3,4824001.88224,5730.03,6500215.0,573003,1747305.0,06037573003,9050.00845755,(POLYGON ((-118.1992330000361 33.7971229999225...
4,6697030.7108,2976.02,6473372.0,297602,1719119.0,06037297602,12308.3153848,(POLYGON ((-118.2879799997457 33.7225829999421...


In [13]:
la_tracts = la_tracts.drop(labels=['shape_area','label','x_center','y_center','shape_len'], axis=1)
la_tracts = la_tracts.rename({'ct10':'tract'}, axis=1)
la_tracts.head()

,tract,geoid10,geometry
0,911001,06037911001,(POLYGON ((-117.6671211134298 34.5580081383781...
1,980003,06037980003,(POLYGON ((-117.8806120004425 34.7636159996687...
2,930301,06037930301,(POLYGON ((-117.6552358388393 34.3972219664572...
3,573003,06037573003,(POLYGON ((-118.1992330000361 33.7971229999225...
4,297602,06037297602,(POLYGON ((-118.2879799997457 33.7225829999421...


In [14]:
la_tracts['tract'] = la_tracts['tract'].astype(int)

In [16]:
la_zero_vehicle = la_tracts.merge(zero_vehicle, on='tract')
la_zero_vehicle = la_zero_vehicle.dropna(subset=['Zero_Vehicle_Percent'])
la_zero_vehicle.head()

,tract,geoid10,geometry,Description,TotalPop,Zero_Vehicle,state,county,Zero_Vehicle_Percent
0,911001,06037911001,(POLYGON ((-117.6671211134298 34.5580081383781...,"Census Tract 9110.01, Los Angeles County, Cali...",1576,48,06,037,3.05
2,930301,06037930301,(POLYGON ((-117.6552358388393 34.3972219664572...,"Census Tract 9303.01, Los Angeles County, Cali...",206,10,06,037,4.85
3,573003,06037573003,(POLYGON ((-118.1992330000361 33.7971229999225...,"Census Tract 5730.03, Los Angeles County, Cali...",592,41,06,037,6.93
4,297602,06037297602,(POLYGON ((-118.2879799997457 33.7225829999421...,"Census Tract 2976.02, Los Angeles County, Cali...",1697,91,06,037,5.36
5,576301,06037576301,(POLYGON ((-118.1850160000802 33.7825970000275...,"Census Tract 5763.01, Los Angeles County, Cali...",1671,328,06,037,19.63


In [17]:
la_zero_vehicle = la_zero_vehicle.drop(labels=['state','county','Description','TotalPop','Zero_Vehicle'], axis=1)la_zero_vehicle = la_zero_vehicle.rename({'tract':'Cenus Tract','geoid10':'FIPS Code', 'geometry':'geometry', 'Zero_Vehicle_Percent':'Percent of Households with No Vehicle'},axis=1)
la_zero_vehicle.head()

,Cenus Tract,FIPS Code,geometry,Percent of Households with No Vehicle
0,911001,06037911001,(POLYGON ((-117.6671211134298 34.5580081383781...,3.05
2,930301,06037930301,(POLYGON ((-117.6552358388393 34.3972219664572...,4.85
3,573003,06037573003,(POLYGON ((-118.1992330000361 33.7971229999225...,6.93
4,297602,06037297602,(POLYGON ((-118.2879799997457 33.7225829999421...,5.36
5,576301,06037576301,(POLYGON ((-118.1850160000802 33.7825970000275...,19.63


In [18]:
la_zero_vehicle.describe()

,Cenus Tract,Percent of Households with No Vehicle
count,2316.000000,2316.000000
mean,402286.601468,9.768230
std,222325.456524,9.357474
min,101110.000000,0.000000
25%,211121.750000,3.427500
50%,404351.500000,6.840000
75%,551226.500000,13.122500
max,980031.000000,86.070000


In [20]:
# save data to a file
# this file can be uploaded to Carto for mapping
la_zero_vehicle.to_file('data/la_zero_vehicle.geojson', driver='GeoJSON')

In [19]:
# this is for reading the data within datahub
viz = ChoroplethViz('data/la_zero_vehicle.geojson', 
                    access_token=token,
                    color_property='Percent of Households with No Vehicle',
                    color_stops=create_color_stops([0, 5, 10, 15,90,100], colors='YlOrRd'),
                    color_function_type='interpolate',
                    line_stroke='-',
                    line_color='grey',
                    line_width=0.25,
                    opacity=0.8,
                    center=(-118.265986,34.036506),
                    zoom=8,
                    below_layer='waterway-label',
                    legend_layout='horizontal',
                    legend_key_shape='bar',
                    legend_key_borders_on=False)
viz.show()

In [21]:
# this is for online viewing -- uploaded to GitHub
viz_online = ChoroplethViz('https://raw.githubusercontent.com/omrachel/equitymetrics/master/la_zero_vehicle.geojson', 
                    access_token=token,
                    color_property='Percent of Households with No Vehicle',
                    color_stops=create_color_stops([0, 5, 10, 15,90,100], colors='YlOrRd'),
                    color_function_type='interpolate',
                    line_stroke='-',
                    line_color='grey',
                    line_width=0.25,
                    opacity=0.8,
                    center=(-118.265986,34.036506),
                    zoom=8,
                    below_layer='waterway-label',
                    legend_layout='horizontal',
                    legend_key_shape='bar',
                    legend_key_borders_on=False)
viz_online.show()

In [23]:
# save html mapc
with open('viz_novehicle.html', 'w') as f:
    f.write(viz_online.create_html())